# 多项式回归示例

## 学习目标

通过本notebook，你将学会：
- 理解多项式回归的原理
- 使用多项式特征处理非线性关系
- 对比不同多项式次数
- 理解过拟合和欠拟合
- 使用正则化防止过拟合

## 课程概述

本notebook将展示如何使用多项式回归处理非线性关系，并演示过拟合现象。


In [ ]:
# 导入必要的库
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 设置随机种子
np.random.seed(42)

# 设置matplotlib在notebook中内联显示
%matplotlib inline

print("环境准备完成！")


## 1. 理论回顾

### 多项式回归的原理

**多项式回归**通过添加多项式特征来处理非线性关系。

**数学形式**：
- 一次：$y = w_1 x + b$
- 二次：$y = w_1 x + w_2 x^2 + b$
- 三次：$y = w_1 x + w_2 x^2 + w_3 x^3 + b$

**关键点**：
- 次数太低：欠拟合，无法捕捉非线性
- 次数太高：过拟合，在训练集好但测试集差
- 需要选择合适的次数，或使用正则化


## 2. 生成非线性数据

生成一个二次关系的数据集（y = x² + 噪声）。


In [ ]:
# 生成非线性数据
np.random.seed(42)
X = np.linspace(-3, 3, 100).reshape(-1, 1)
# 二次关系：y = x^2 + noise
y = X.flatten() ** 2 + np.random.randn(100) * 0.5

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"数据信息:")
print(f"  训练集大小: {X_train.shape[0]}")
print(f"  测试集大小: {X_test.shape[0]}")
print(f"  数据关系: y = x² + noise (非线性)")

# 可视化数据
plt.figure(figsize=(8, 6))
plt.scatter(X_train, y_train, alpha=0.6, label='训练数据', s=30)
plt.scatter(X_test, y_test, alpha=0.6, label='测试数据', marker='x', s=30)
plt.xlabel('X', fontsize=12)
plt.ylabel('y', fontsize=12)
plt.title('非线性数据分布', fontsize=14)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.show()


## 3. 对比不同多项式次数

对比不同多项式次数的拟合效果。


In [ ]:
degrees = [1, 2, 3, 5, 10, 20]
results = {}

print("=" * 70)
print("多项式次数对比")
print("=" * 70)
print(f"\n{'次数':<10} {'训练MSE':<15} {'测试MSE':<15} {'R²':<15}")
print("-" * 55)

for degree in degrees:
    # 创建多项式特征
    poly_features = PolynomialFeatures(degree=degree, include_bias=False)
    X_train_poly = poly_features.fit_transform(X_train)
    X_test_poly = poly_features.transform(X_test)
    
    # 训练模型
    model = LinearRegression()
    model.fit(X_train_poly, y_train)
    
    # 预测
    y_pred_train = model.predict(X_train_poly)
    y_pred_test = model.predict(X_test_poly)
    
    # 评估
    train_mse = mean_squared_error(y_train, y_pred_train)
    test_mse = mean_squared_error(y_test, y_pred_test)
    test_r2 = r2_score(y_test, y_pred_test)
    
    results[degree] = {
        'model': model,
        'poly_features': poly_features,
        'train_mse': train_mse,
        'test_mse': test_mse,
        'test_r2': test_r2,
        'y_pred_test': y_pred_test
    }
    
    print(f"{degree:<10} {train_mse:<15.4f} {test_mse:<15.4f} {test_r2:<15.4f}")

print("\n观察：")
print("- degree=1（线性）：欠拟合，无法捕捉非线性")
print("- degree=2（二次）：最佳，符合真实关系")
print("- degree=10, 20：过拟合，测试MSE增大")


## 4. 可视化不同次数的拟合结果

可视化不同多项式次数的拟合曲线。


In [ ]:
n_degrees = len(results)
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

X_plot = np.linspace(X_train.min(), X_train.max(), 200).reshape(-1, 1)

for idx, (degree, result) in enumerate(results.items()):
    ax = axes[idx]
    
    # 数据点
    ax.scatter(X_train, y_train, alpha=0.5, label='训练数据', s=20)
    ax.scatter(X_test, y_test, alpha=0.5, label='测试数据', s=20, marker='x')
    
    # 拟合曲线
    X_plot_poly = result['poly_features'].transform(X_plot)
    y_plot = result['model'].predict(X_plot_poly)
    ax.plot(X_plot, y_plot, 'r-', linewidth=2, label=f'拟合曲线')
    
    ax.set_xlabel('X', fontsize=10)
    ax.set_ylabel('y', fontsize=10)
    ax.set_title(f'次数={degree}, R²={result["test_r2"]:.4f}\n测试MSE={result["test_mse"]:.4f}', fontsize=11)
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 5. 过拟合演示

使用高次多项式演示过拟合现象，并展示如何使用正则化防止过拟合。


In [ ]:
# 使用高次多项式（容易过拟合）
degree = 15
poly_features = PolynomialFeatures(degree=degree, include_bias=False)
X_train_poly = poly_features.fit_transform(X_train)
X_test_poly = poly_features.transform(X_test)

# 普通线性回归（可能过拟合）
model_no_reg = LinearRegression()
model_no_reg.fit(X_train_poly, y_train)

# Ridge回归（防止过拟合）
model_ridge = Ridge(alpha=0.1)
model_ridge.fit(X_train_poly, y_train)

# 评估
y_pred_train_no_reg = model_no_reg.predict(X_train_poly)
y_pred_test_no_reg = model_no_reg.predict(X_test_poly)
y_pred_train_ridge = model_ridge.predict(X_train_poly)
y_pred_test_ridge = model_ridge.predict(X_test_poly)

train_mse_no_reg = mean_squared_error(y_train, y_pred_train_no_reg)
test_mse_no_reg = mean_squared_error(y_test, y_pred_test_no_reg)
train_mse_ridge = mean_squared_error(y_train, y_pred_train_ridge)
test_mse_ridge = mean_squared_error(y_test, y_pred_test_ridge)

print(f"\n高次多项式 (degree={degree}):")
print(f"{'模型':<20} {'训练MSE':<15} {'测试MSE':<15} {'差距':<15}")
print("-" * 65)
print(f"{'无正则化':<20} {train_mse_no_reg:<15.4f} {test_mse_no_reg:<15.4f} "
      f"{test_mse_no_reg - train_mse_no_reg:<15.4f}")
print(f"{'Ridge回归':<20} {train_mse_ridge:<15.4f} {test_mse_ridge:<15.4f} "
      f"{test_mse_ridge - train_mse_ridge:<15.4f}")

# 可视化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

X_plot = np.linspace(X_train.min(), X_train.max(), 200).reshape(-1, 1)
X_plot_poly = poly_features.transform(X_plot)

for idx, (model, title) in enumerate([(model_no_reg, '无正则化（过拟合）'),
                                      (model_ridge, 'Ridge回归（防止过拟合）')]):
    ax = axes[idx]
    
    ax.scatter(X_train, y_train, alpha=0.5, label='训练数据', s=20)
    ax.scatter(X_test, y_test, alpha=0.5, label='测试数据', s=20, marker='x')
    
    y_plot = model.predict(X_plot_poly)
    ax.plot(X_plot, y_plot, 'r-', linewidth=2, label='拟合曲线')
    
    ax.set_xlabel('X', fontsize=12)
    ax.set_ylabel('y', fontsize=12)
    ax.set_title(title, fontsize=14)
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n观察：")
print("- 无正则化：训练MSE很小，但测试MSE很大（过拟合）")
print("- Ridge回归：训练和测试MSE接近（防止过拟合）")


## 6. 总结

### 关键知识点

1. **多项式回归**：
   - 可以处理非线性关系
   - 通过添加多项式特征实现

2. **次数选择**：
   - 太低：欠拟合
   - 太高：过拟合
   - 需要根据数据特点选择

3. **过拟合问题**：
   - 高次多项式容易过拟合
   - 可以使用正则化防止过拟合

4. **正则化**：
   - Ridge回归可以有效防止过拟合
   - 在保持拟合能力的同时提高泛化能力

### 选择建议

- **二次关系**：degree=2通常是最佳选择
- **复杂关系**：使用较高次数+正则化
- **简单关系**：使用较低次数

---

**恭喜完成多项式回归的学习！** 🎉
